# സെഷൻ 6 – മൾട്ടി-സ്റ്റെപ്പ് പൈപ്പ്‌ലൈൻ (പ്ലാൻ → എക്സിക്യൂട്ട് → റിഫൈൻ)

ഓരോ ഘട്ടത്തിനും മോഡലുകൾ തിരഞ്ഞെടുക്കാൻ റൂട്ടിംഗ് ഫംഗ്ഷനുകൾ ഉപയോഗിച്ച്, പിന്നീട് അന്തിമ ഉത്തരം മെച്ചപ്പെടുത്തുന്നു.


### വിശദീകരണം: ആശ്രിത ഇൻസ്റ്റലേഷൻ
പ്രതിസ്റ്റെപ്പ് ചാറ്റ് കോൾസിനായി ആവശ്യമായ `foundry-local-sdk` ഉം `openai` ഉം ഇൻസ്റ്റാൾ ചെയ്യുന്നു. വീണ്ടും പ്രവർത്തിപ്പിക്കാനും സുരക്ഷിതമാണ്.


# സീനാരിയോ
ബഹു-പടി മോഡൽ പൈപ്പ്‌ലൈൻ: (1) ഒരു ടാസ്ക് വ്യത്യസ്ത ഘട്ടങ്ങളായി പദ്ധതിയിടുന്നു, (2) ഉദ്ദേശ്യാധിഷ്ഠിത മോഡൽ തിരഞ്ഞെടുപ്പോടെ ഓരോ ഘട്ടവും നടപ്പിലാക്കുന്നു, (3) അന്തിമ സംയോജിത ഉത്തരം മെച്ചപ്പെടുത്തുന്നു. ഗുണനിലവാരം നിലനിർത്തിക്കൊണ്ട് കാര്യക്ഷമതയ്ക്കായി വ്യത്യസ്ത SLM കഴിവുകൾ ചൈനിംഗ് ചെയ്യുന്നത് പ്രദർശിപ്പിക്കുന്നു.


In [13]:
!pip install -q foundry-local-sdk openai

### Explanation: Core Imports
ഇന്റന്റ് ഡിറ്റക്ഷനിനായി regex, ഓരോ അലിയാസിനും അനുബന്ധിപ്പിക്കാൻ Foundry Local മാനേജർ, ചാറ്റ് പൂർത്തീകരണങ്ങൾക്ക് OpenAI ക്ലയന്റ് എന്നിവ ഇറക്കുമതി ചെയ്യുന്നു.


In [14]:
import re
from foundry_local import FoundryLocalManager
from openai import OpenAI

### വിശദീകരണം: കഴിവ് കാറ്റലോഗും നിയമങ്ങളും
കഴിവ്-അറിയുന്ന കാറ്റലോഗും സ്റ്റെപ്പ് ടെക്സ്റ്റ് ഉദ്ദേശ്യ വിഭാഗങ്ങളിലേക്ക് (കോഡ്, സംഗ്രഹം, വർഗ്ഗീകരണം, പൊതുവായ) മാപ്പ് ചെയ്യാൻ ഉപയോഗിക്കുന്ന regex നിയമങ്ങളും നിർവചിക്കുന്നു. ഒരേ ഉദ്ദേശ്യം പിന്തുണയ്ക്കുന്ന നിരവധി മോഡലുകൾ ഉണ്ടെങ്കിൽ കുറഞ്ഞ മുൻഗണന മൂല്യങ്ങൾ തുല്യതകൾ ജയിക്കും.


In [15]:
CATALOG = {
 'phi-4-mini': {'capabilities':['general','summarize'],'priority':2},
 'qwen2.5-coder-7b': {'capabilities':['code','refactor'],'priority':1},
 'qwen2.5-0.5b': {'capabilities':['classification','fast'],'priority':3},
}
RULES = [
 (re.compile('code|refactor|function', re.I), 'code'),
 (re.compile('summari|abstract|tl;dr', re.I), 'summarize'),
 (re.compile('classif|category|label', re.I), 'classification'),
]

### വിശദീകരണം: ഉദ്ദേശ്യം, മോഡൽ തിരഞ്ഞെടുപ്പ് & ചാറ്റ് സഹായി
നൽകുന്നു:
- regex അടിസ്ഥാനത്തിലുള്ള വർഗ്ഗീകരണത്തിനായി `detect_intent`.
- കഴിവ് + മുൻഗണന പ്രകാരം മികച്ച ഉപനാമം തിരഞ്ഞെടുക്കാൻ `pick_model`.
- ഓരോ ഉപനാമത്തിനും ഒരു ക്ലയന്റ് സൃഷ്ടിച്ച് ഒറ്റ തവണയുടെ പ്രതികരണം നൽകുന്ന സൗകര്യപ്രദമായ `chat` റാപ്പർ.


In [16]:
def detect_intent(text: str):
    """Return an intent label based on simple regex rules; falls back to 'general'."""
    for pat, intent in RULES:
        if pat.search(text):
            return intent
    return 'general'

def pick_model(intent: str) -> str:
    """Pick the best model for an intent using capability match first, then priority."""
    ranked = []
    for name, meta in CATALOG.items():
        ranked.append((name, intent in meta['capabilities'], meta['priority']))
    # Sort: capability match (True first), then lower priority value
    ranked.sort(key=lambda t: (not t[1], t[2]))
    return ranked[0][0]

def chat(alias: str, content: str, temp: float = 0.4) -> str:
    """Simple helper to send a single user message to a Foundry Local model via OpenAI client."""
    m = FoundryLocalManager(alias)
    client = OpenAI(base_url=m.endpoint, api_key=m.api_key or 'not-needed')
    mid = m.get_model_info(alias).id
    resp = client.chat.completions.create(
        model=mid,
        messages=[{'role': 'user', 'content': content}],
        max_tokens=180,
        temperature=temp,
    )
    return resp.choices[0].message.content

### Explanation: Multi-Step Pipeline Function
പൈപ്പ്‌ലൈൻ നടപ്പിലാക്കുന്നു: പദ്ധതി → ഘട്ടങ്ങൾ പാഴ്‌സ് ചെയ്യുക → ഉദ്ദേശ്യാധിഷ്ഠിത റൂട്ടിംഗോടെ ഓരോ ഘട്ടവും നടപ്പിലാക്കുക → സംയുക്ത ഔട്ട്പുട്ടുകൾ മെച്ചപ്പെടുത്തുക. പരിശോധനയ്ക്കോ മൂല്യനിർണയത്തിനോ ഘടനയുള്ള ഡിക്ഷണറി മടക്കുന്നു.


In [17]:
def pipeline(task: str):
    """Multi-step pipeline: plan → execute steps → refine final answer.

    Returns dict with keys: plan (raw plan text), steps (list of tuples), final (refined answer).
    Each step tuple: (index, step_text, step_result, model_alias_used)
    """
    # 1. Plan
    plan_alias = pick_model('general')
    plan_prompt = (
        "Break the task into 3 concise, actionable steps (no extra commentary).\n"
        f"Task: {task}"
    )
    plan = chat(plan_alias, plan_prompt)

    # 2. Parse steps (robust to numbering or bullet styles)
    raw_lines = [l.strip() for l in plan.splitlines() if l.strip()]
    steps = []
    for line in raw_lines:
        cleaned = re.sub(r'^\d+[).:-]?\s*', '', line)  # remove leading numbering
        cleaned = re.sub(r'^[-*]\s*', '', cleaned)      # remove bullet markers
        if cleaned:
            steps.append(cleaned)
        if len(steps) == 3:
            break
    if not steps:
        steps = [task]

    # 3. Execute steps
    outputs = []
    for idx, step in enumerate(steps, 1):
        intent = detect_intent(step)
        exec_alias = pick_model(intent)
        exec_prompt = (
            f"Execute step {idx} for the overall task.\n"
            f"Overall task: {task}\n"
            f"Step {idx}: {step}\n"
            "Return a concise result focusing only on the step objective."
        )
        result = chat(exec_alias, exec_prompt)
        outputs.append((idx, step, result, exec_alias))

    # 4. Refine final answer
    refine_alias = pick_model('summarize')
    combined = "\n\n".join(
        f"Step {idx} ({alias}) Output:\n{res}" for idx, _step, res, alias in outputs
    )
    refine_prompt = (
        "You are a senior assistant. Synthesize these step outputs into a cohesive final answer.\n"
        "Ensure clarity, avoid repetition, and highlight improvements or key insights if relevant.\n\n"
        f"Task: {task}\n\n"
        f"Step Outputs:\n{combined}"
    )
    final_ans = chat(refine_alias, refine_prompt)

    return {"plan": plan, "steps": outputs, "final": final_ans}

### Explanation: Run Example Task
കോഡ് + സംഗ്രഹം എന്ന മിശ്രിത ഉദ്ദേശ്യങ്ങൾ കാണിക്കുന്ന ഒരു പുനഃസംഘടന-കേന്ദ്രിത ടാസ്കിൽ പൂർണ്ണ പൈപ്പ്‌ലൈൻ പ്രവർത്തിപ്പിക്കുന്നു. ഫലം ഡിക്ഷണറി റോ പ്ലാൻ, ഓരോ ഘട്ടത്തിന്റെയും ഔട്ട്പുട്ടുകൾ തിരഞ്ഞെടുക്കപ്പെട്ട മോഡൽ ഉപനാമങ്ങളോടുകൂടി, അന്തിമ സംയോജിത ഉത്തരം എന്നിവ കാണിക്കുന്നു.


In [18]:
result = pipeline('Generate a refactored version of a slow Python loop and summarize performance gains.')
result

{'plan': '1. Profile the existing slow Python loop to identify bottlenecks.\n2. Refactor the loop using optimized techniques (e.g., list comprehensions, map, or itertools).\n3. Compare the performance of the refactored loop with the original using a benchmarking tool and summarize the gains.',
 'steps': [(1,
   'Profile the existing slow Python loop to identify bottlenecks.',
   "To execute step 1, you would use a profiling tool like `cProfile` in Python to analyze the performance of the existing slow loop. Here's an example of how you might do this:\n\n```python\nimport cProfile\n\ndef slow_loop():\n    # Example of a slow loop\n    result = []\n    for i in range(1000000):\n        result.append(i * i)\n    return result\n\n# Profile the slow loop\ncProfile.run('slow_loop()', 'profile_stats')\n\n# To see the results, you can use pstats module\nimport pstats\np = pstats.Stats('profile_stats')\np.sort_stats('cumulative').print_stats()\n```\n\nThis code will run the `slow_loop` function

### Explanation: Display Result Object
വേഗത്തിലുള്ള പരിശോധനയ്ക്കോ താഴെനിന്നുള്ള മൂല്യനിർണയത്തിനോ (ഉദാ: ഘട്ട ഗുണനിലവാരം അളക്കൽ അല്ലെങ്കിൽ മെച്ചപ്പെടുത്തൽ ഫലപ്രാപ്തി) ഘടനാപരമായ പൈപ്പ്‌ലൈൻ ഔട്ട്പുട്ട് കാണിക്കുന്നു.


---

<!-- CO-OP TRANSLATOR DISCLAIMER START -->
**അസൂയാപത്രം**:  
ഈ രേഖ AI വിവർത്തന സേവനം [Co-op Translator](https://github.com/Azure/co-op-translator) ഉപയോഗിച്ച് വിവർത്തനം ചെയ്തതാണ്. നാം കൃത്യതയ്ക്ക് ശ്രമിച്ചിട്ടുണ്ടെങ്കിലും, യന്ത്രം ചെയ്ത വിവർത്തനങ്ങളിൽ പിശകുകൾ അല്ലെങ്കിൽ തെറ്റുകൾ ഉണ്ടാകാമെന്ന് ദയവായി ശ്രദ്ധിക്കുക. അതിന്റെ മാതൃഭാഷയിലുള്ള യഥാർത്ഥ രേഖ അധികാരപരമായ ഉറവിടമായി കണക്കാക്കണം. നിർണായകമായ വിവരങ്ങൾക്ക്, പ്രൊഫഷണൽ മനുഷ്യ വിവർത്തനം ശുപാർശ ചെയ്യപ്പെടുന്നു. ഈ വിവർത്തനം ഉപയോഗിക്കുന്നതിൽ നിന്നുണ്ടാകുന്ന ഏതെങ്കിലും തെറ്റിദ്ധാരണകൾക്കോ തെറ്റായ വ്യാഖ്യാനങ്ങൾക്കോ ഞങ്ങൾ ഉത്തരവാദികളല്ല.
<!-- CO-OP TRANSLATOR DISCLAIMER END -->
